# Context Manager
The context manager is a Python class designed to process batches of legal documents through the LLMSummarizer, preparing the data for other applications. It iterates over each section in a dictionary, summarizes the values, and returns the results.

In [34]:
from llm_context_manager import LLMContextManager, dict_to_text
from llm_analyzer import LLMAnalyzer, extract_args
import openai
from termcolor import colored
from utils import parse_string_to_list

In [40]:
openai.api_key="use-your-open-ai-key-here"

### The Setup
We're going to mock a vectordata base query by loading flat files.

In [8]:
# Load all files from the 'examples' directory
llm_context_manager = LLMContextManager.load_from_directory(dir_path='examples')

loading doc9.txt
loading .DS_Store
loading doc8.txt
loading doc12.txt
loading doc10.txt
loading doc11.txt
loading doc6.txt
loading doc7.txt
loading doc5.txt
loading doc4.txt
loading doc1.txt
loading doc3.txt
loading doc2.txt


### Explore
Let's take a look at one example data.

Note that these are not real legal documents.  I generated this data for demo purposes using GPT4.

In [9]:
# Print out one of the loaded sections
print(f"Contents of the 'example_section' section: {llm_context_manager.sections.get('doc1', 'Section not found.')}")
print(f"llm_prompt.get_section_keys(): {llm_context_manager.get_section_keys()}")

Contents of the 'example_section' section: AUTO LOAN AGREEMENT
THIS AGREEMENT is made this 1st day of March, 2023.

BETWEEN:
ABC Auto Loans 157, a financial institution duly organized and existing under the laws of the state, having its principal place of business at 2793 Elm St, Springfield, USA (hereinafter referred to as the "Lender").

John Doe 951, an individual residing at 512 Main St, Springfield, USA (hereinafter referred to as the "Borrower").

RECITALS:
WHEREAS, the Borrower has requested the Lender to extend a loan to the Borrower for the purpose of financing the acquisition of a vehicle described in detail in ARTICLE 1;

WHEREAS, the Lender has agreed to extend such a loan to the Borrower on the terms and conditions set forth herein;

NOW, THEREFORE, in consideration of the mutual covenants contained herein and other good and valuable consideration, the receipt and sufficiency of which are hereby acknowledged, the parties hereto agree as follows:

ARTICLE 1 – LOAN DETAILS
1

## Total context length
There's 8931 words in the total corpus, which is definitely over the 4097 token limit for the text-davinci-003 model.

In [10]:
#: Get the word count:
total_words = llm_context_manager.get_total_word_count()
print(f"The total number of words in the values is: {total_words}")

The total number of words in the values is: 8931


## Summarize 
Apply the summarizer to extract key points.

In [26]:
#: Get summarized content:
summarized_context = dict_to_text(llm_context_manager.get_summarized_context())
print(f"\nsummarized_context: \n {summarized_context}")


summarized_context: 
 Document ID: doc9
Lender: VWX Auto Financing 411
Borrower: Taylor Green 361
Loan Amount: $38,267
Interest Rate: 11.49% 
Down Payment: $2,430.43
Loan Term: 57 months

Document ID: doc8
Lender: STU Car Loans 560
Borrower: Alex Lee 190
Loan Amount: $71,623
Interest Rate: 9.96%
Down Payment: $11,835.52
Loan Term: 70 months

Document ID: doc12
Lender: ST Auto Loans 680
Borrower: Pat Jordan 945
Loan Amount: $44,973
Interest Rate: 4.06%
Down Payment: $7,913.52
Loan Term: 47 months

Document ID: doc10
Output: 
Lender: YZ Auto Lenders 684
Borrower: Jordan Brown 809
Loan Amount: $73,905
Interest Rate: 6.89%
Down Payment: $3,344.81
Loan Term: 49 months

Document ID: doc11
Lender: QR Auto Finance 268
Borrower: Casey White 541
Loan Amount: US$12,404
Interest Rate: 1.57%
Down Payment: US$908.06
Loan Term: 50 months

Document ID: doc6
Lender: MNO Auto Lenders 589
Borrower: Jordan Brown 106
Loan Amount: $23,507
Interest Rate: 9.59%
Down Payment: $1,439.31
Loan Term: 73 months

D

## Analyze the summarized key points
Now, we'll apply the analyzer on the summarized key points.  Here, we use openai function calling to provide the LLM with a basic tool that allows it to sum up loan amounts over the summarized context.

In [19]:
llm_analyzer = LLMAnalyzer()

self.prompt: Using the available data, extract the debt amounts and pass them into the available function to compute their sum. They should be designated as Loan Amount:


In [28]:
#: Call the fuction
function_call = llm_analyzer.do_analysis(summarized_context)
print(colored(f"function_call: {function_call}", "green"))

full_message: {
  "index": 0,
  "message": {
    "role": "assistant",
    "content": null,
    "function_call": {
      "name": "compute_total_debt",
      "arguments": "{\n\"debts\": [38267, 71623, 44973, 73905, 12404, 23507, 14488, 9826, 17706, 52552, 14679, 53178]\n}"
    }
  },
  "finish_reason": "stop"
}
function_call: {
  "index": 0,
  "message": {
    "role": "assistant",
    "content": null,
    "function_call": {
      "name": "compute_total_debt",
      "arguments": "{\n\"debts\": [38267, 71623, 44973, 73905, 12404, 23507, 14488, 9826, 17706, 52552, 14679, 53178]\n}"
    }
  },
  "finish_reason": "stop"
}


## Handling the debt amounts extracted by the LLM

We still have to parse the function arguments from string form to python objects.

The arguments represent a list of debts extracted by our LLMSummarizer, by way of the LLMContextManager.

In [36]:
function_args = function_call['message']['function_call']['arguments']
args = extract_args(function_args)
args = parse_string_to_list(args)

print(type(args))
print(colored(f"args: {args}", "blue"))

s: {
"debts": [38267, 71623, 44973, 73905, 12404, 23507, 14488, 9826, 17706, 52552, 14679, 53178]
}
type(s): <class 'str'>
<class 'list'>
args: [38267, 71623, 44973, 73905, 12404, 23507, 14488, 9826, 17706, 52552, 14679, 53178]


### Final Result.
Now, we can pass the function arguments to the python function to compute the total debt.

In [37]:
#: The actual python function that will be called using the llm output
def compute_total_debt(inputs):
    return sum(inputs)

In [38]:
final_result = compute_total_debt(args)
print(f"final_result: {final_result}")

final_result: 427108


# Final Answer: $427,108.00